
   1 - Preparar o ambiente de desenvolvimento:
        Crie um ambiente virtual para seu projeto utilizando uma ferramenta como venv, virtualenv ou pipenv.
        Instale as bibliotecas necessárias (Streamlit, statsbombpy, mplsoccer, matplotlib, entre outras).



   2 - Estruturar o projeto:
        Crie um repositório no GitHub para hospedar o código do seu projeto.
        Garanta que o repositório contenha um arquivo requirements.txt com as dependências necessárias para rodar o projeto.
        Organize o código de forma clara, criando funções separadas para carregar os dados, gerar as visualizações e construir a interface do dashboard.


In [4]:
import streamlit as st
import pandas as pd
from statsbombpy import sb
from mplsoccer import Pitch, VerticalPitch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns


In [2]:
def load_competitions():
    return sb.competitions()

def load_matches(comp_id, season_id):
    return sb.matches(competition_id=comp_id, season_id=season_id)

def load_events(match_id):
    return sb.events(match_id=match_id)

   3-  Definir a estrutura do dashboard:
        Desenvolva uma interface interativa em Streamlit que permita ao usuário selecionar:
            Um campeonato específico.
            Uma temporada (ano).
            Uma partida ou jogador para análise.
        Organize o layout do dashboard em colunas, usando columns, sidebars, containers e tabs para melhorar a usabilidade.


In [ ]:

# Título do App
st.title('Análise de Dados do StatsBomb com Visualização de Campo')

# Seção 1: Competitions
st.subheader('1. Competições Disponíveis')
competitions = sb.competitions()
st.dataframe(competitions)

# Selecionar Competição e Temporada
st.sidebar.subheader('Selecione a Competição e a Temporada')
competition_id = st.sidebar.selectbox(
    'Escolha a Competição',
    competitions['competition_name'].unique()
)

season_id = st.sidebar.selectbox(
    'Escolha a Temporada',
    competitions[competitions['competition_name'] == competition_id]['season_name'].unique()
)

# Obter IDs
competition_selected = competitions[competitions['competition_name'] == competition_id]
season_selected = competition_selected[competition_selected['season_name'] == season_id]
comp_id = int(season_selected['competition_id'])
season_id = int(season_selected['season_id'])

# Seção 2: Matches
st.sidebar.subheader('Selecione a Partida')
matches = sb.matches(competition_id=comp_id, season_id=season_id)

# Ordenar partidas por data
matches = matches.sort_values(by='match_date')

# Formatar as opções de seleção com a data da partida
match_id = st.sidebar.selectbox(
    'Escolha a Partida',
    matches['match_id'].unique(),
    format_func=lambda x: f"{matches[matches['match_id'] == x]['home_team'].values[0]} vs {matches[matches['match_id'] == x]['away_team'].values[0]} - {matches[matches['match_id'] == x]['match_date'].values[0]}"
)


# Seção 3: Lineups
team_lineup = sb.lineups(match_id=match_id)

# Criar uma lista de todos os jogadores disponíveis
all_players = []
for team, lineup in team_lineup.items():
    st.write(f'**{team}**')
    st.dataframe(lineup)
    all_players.extend(lineup['player_name'].tolist())



   4 - Obter dados e exibir informações básicas
        Use a biblioteca StatsBombPy para carregar dados de competições, temporadas, partidas e jogadores.
        Mostre, em uma página do dashboard, as seguintes informações:
            Nome da competição, temporada e partida selecionada.
            Estatísticas básicas da partida (gols, chutes, passes, etc.).
            Um DataFrame exibindo os eventos da partida, como passes, finalizações e desarmes.


In [ ]:
# Exibir Nome da Competição, Temporada e Partida Selecionada
st.header(f"Competição: {competition_id}")
st.subheader(f"Temporada: {season_id}")
match_details = matches[matches['match_id'] == match_id].iloc[0]
st.subheader(f"Partida Selecionada: {match_details['home_team']} vs {match_details['away_team']}")
st.subheader(f'Escalações - Jogo {match_id}')

# Seção 4: Eventos da Partida
st.header('Eventos da Partida')

# Carregar eventos da partida
events = sb.events(match_id=match_id)

# Filtrar eventos principais
main_event_types = ['Pass', 'Shot', 'Duel', 'Foul', 'Clearance']
filtered_events = events[events['type'].isin(main_event_types)]

# Exibir um DataFrame com os eventos filtrados
st.write(f"Eventos principais da partida ({', '.join(main_event_types)})")
st.dataframe(filtered_events[['minute', 'second', 'type', 'player', 'team', 'location', 'pass_outcome', 'shot_outcome']])

# Seção 5: Seleção de Jogador para Visualização
st.sidebar.subheader('Selecione um Jogador para Visualizar Movimentação')
player_name = st.sidebar.selectbox('Escolha o jogador para análise', all_players)

# Verificar se a coluna 'player' existe e usá-la
if 'player' in events.columns:
    player_column = 'player'
else:
    st.error('A coluna "player" não foi encontrada nos dados de eventos.')
    player_column = None

if player_column:
    # Filtrar eventos pelo jogador selecionado
    player_events = events[events[player_column] == player_name]
    
    # Verificar se há eventos com localização válida
    valid_events = player_events[player_events['location'].apply(lambda x: isinstance(x, list) and len(x) == 2)]

    if valid_events.empty:
        st.warning(f'Nenhum evento com localização válida encontrado para {player_name}.')
    else:
        st.subheader(f'Eventos de {player_name}')
        st.dataframe(valid_events)




   5 - Criar visualizações de dados
        Utilize a biblioteca mplsoccer para gerar um mapa de passes e mapa de chutes para uma partida específica. Garanta que o gráfico seja interativo, com legendas e informações que ajudem a interpretar os dados.
        Crie visualizações adicionais com Matplotlib e Seaborn para explorar relações entre as estatísticas de uma partida ou de um jogador (por exemplo, relação entre número de passes e gols).
        Utilize a biblioteca mplsoccer para novas visualizações de acordo com sua galeria (https://mplsoccer.readthedocs.io/en/latest/gallery/index.html)


In [ ]:
# Seção 5: Gráficos de Dados
st.header('Visualizações Gráficas dos Eventos')

# Seção 4: Controle de Tempo (Faixa de Minutos)
st.sidebar.subheader('Controle de Tempo')

# Filtrar o tempo máximo da partida para o slider
max_minute = int(filtered_events['minute'].max())

# Slider de faixa de tempo para selecionar intervalo mínimo e máximo
selected_minute_range = st.sidebar.slider(
    'Selecione o Intervalo de Minutos da Partida',
    0, max_minute, (0, max_minute)  # Padrão: toda a partida
)

# Filtrar eventos dentro do intervalo de minutos selecionado
filtered_events_in_range = filtered_events[
    (filtered_events['minute'] >= selected_minute_range[0]) &
    (filtered_events['minute'] <= selected_minute_range[1])
]

# Limpar os gráficos a cada nova seleção de tempo
plt.clf()

# Seção 5: Mapa de Passes em Sequência Temporal
st.header('Mapa de Passes em Sequência Temporal')

# Filtrar passes
passes = filtered_events_in_range[filtered_events_in_range['type'] == 'Pass']

# Criar o gráfico de campo
pitch = Pitch(pitch_type='statsbomb', line_zorder=2, pitch_color='#22312b', line_color='white')
fig, ax = pitch.draw(figsize=(10, 7))

# Plotar passes dentro do intervalo selecionado
for i, row in passes.iterrows():
    if isinstance(row['location'], list) and isinstance(row['pass_end_location'], list):
        pitch.arrows(row['location'][0], row['location'][1],
                    row['pass_end_location'][0], row['pass_end_location'][1],
                    ax=ax, width=2, headwidth=3, color='blue', alpha=0.6)
        # Adicionar legenda com o jogador e o minuto do passe
        ax.annotate(f"{row['player']} ({row['minute']}')", (row['location'][0], row['location'][1]), color='white', fontsize=8)

# Exibir o gráfico
st.pyplot(fig)

# Seção 6: Mapa de Chutes
st.header('Mapa de Chutes')

# Filtrar chutes
shots = filtered_events_in_range[filtered_events_in_range['type'] == 'Shot']

# Criar o gráfico de campo para chutes
fig, ax = pitch.draw(figsize=(10, 7))

# Plotar os chutes dentro do intervalo selecionado
for i, row in shots.iterrows():
    if isinstance(row['location'], list):
        pitch.scatter(row['location'][0], row['location'][1], ax=ax, s=100, color='red', edgecolor='black', alpha=0.7)
        # Adicionar legenda com o jogador e o minuto do chute
        ax.annotate(f"{row['player']} ({row['minute']}')", (row['location'][0], row['location'][1]), color='white', fontsize=8)

# Exibir o gráfico
st.pyplot(fig)

# Seção 7: Mapa de Desarmes
st.header('Mapa de Desarmes')

# Filtrar desarmes (Duel event type)
duels = filtered_events_in_range[filtered_events_in_range['type'] == 'Duel']

# Criar o gráfico de campo para desarmes
fig, ax = pitch.draw(figsize=(10, 7))

# Plotar os desarmes dentro do intervalo selecionado
for i, row in duels.iterrows():
    if isinstance(row['location'], list):
        pitch.scatter(row['location'][0], row['location'][1], ax=ax, s=100, color='green', edgecolor='black', alpha=0.7)
        # Adicionar legenda com o jogador e o minuto do desarme
        ax.annotate(f"{row['player']} ({row['minute']}')", (row['location'][0], row['location'][1]), color='white', fontsize=8)

# Exibir o gráfico
st.pyplot(fig)


# Visualização de Chutes
if 'Shot' in filtered_events['type'].values:
    st.subheader("Distribuição de Chutes")
    shots = filtered_events[filtered_events['type'] == 'Shot']

# Criar o campo com direção vertical para melhor visualização dos chutes
pitch = VerticalPitch(pitch_type='statsbomb', half=True, pitch_color='#22312b', line_color='white')
fig, ax = pitch.draw(figsize=(10, 7))

# Plotar os chutes no campo
for i, row in shots.iterrows():
    x_start, y_start = row['location']
    ax.scatter(x_start, y_start, color='red', s=100, edgecolors='black')

ax.set_title('Distribuição de Chutes', fontsize=20)
st.pyplot(fig)

# Seção 6: 360 Frames (Se disponível)
st.subheader(f'6. Frames 360° - Jogo {match_id}')
if st.sidebar.checkbox('Incluir 360 Frames', value=False):
    frames = sb.frames(match_id=match_id)
    st.dataframe(frames.head(10))


# Seção 8: DataFrame com os Eventos no Intervalo de Tempo Selecionado
st.subheader('Dados dos Eventos no Intervalo de Tempo Selecionado')
st.dataframe(filtered_events_in_range[['minute', 'type', 'player', 'team', 'location']])

In [ ]:
# Criar gráfico de campo
pitch = Pitch(pitch_type='statsbomb', line_zorder=2, pitch_color='#22312b', line_color='white')
fig, ax = pitch.draw(figsize=(10, 7))

# Plotar passes
passes = filtered_events_in_range[filtered_events_in_range['type'] == 'Pass']
for i, row in passes.iterrows():
    pitch.arrows(row['location'][0], row['location'][1], row['pass_end_location'][0], row['pass_end_location'][1],
                 ax=ax, width=2, headwidth=3, color='blue', alpha=0.6)
    ax.annotate(f"{row['player']} ({row['minute']}')", (row['location'][0], row['location'][1]), color='white', fontsize=8)

st.pyplot(fig)


   6 - Adicionar interatividade
        Adicione seletores de jogadores e botões de filtro que permitam ao usuário visualizar apenas eventos relacionados a um jogador específico.
        Inclua botões de download que permitam ao usuário baixar os dados filtrados da partida em formato CSV.
        Utilize barras de progresso e spinners para informar ao usuário que os dados estão sendo carregados ou processados.


In [ ]:
# Selecionar jogador
player_name = st.sidebar.selectbox('Escolha o jogador para análise', all_players)

# Filtrar eventos pelo jogador
player_events = events[events['player'] == player_name]
st.dataframe(player_events)

# Botão de download dos dados filtrados
csv = player_events.to_csv(index=False)
st.download_button("Baixar Dados Filtrados", csv, "player_events.csv")


   7 - Incluir métricas e indicadores
        Exiba indicadores numéricos usando a função metric() do Streamlit para mostrar, por exemplo:
            Total de gols da partida.
            Quantidade de passes bem-sucedidos de um jogador.
            Taxa de conversão de chutes em gol.
        Personalize esses indicadores com cores que realcem os valores mais importantes.


In [ ]:
# Métricas
total_goals = len(events[events['shot_outcome'] == 'Goal'])
successful_passes = len(passes[passes['pass_outcome'] == 'Complete'])
shots = len(shots)
conversion_rate = total_goals / shots if shots > 0 else 0

# Exibir métricas
st.metric("Total de Gols", total_goals)
st.metric("Passes Bem-Sucedidos", successful_passes)
st.metric("Taxa de Conversão de Chutes", f"{conversion_rate:.2%}")


   8 - Criar formulários interativos
        Desenvolva formulários simples que permitam ao usuário escolher, por exemplo, a quantidade de eventos a serem visualizados, o intervalo de tempo de uma partida ou a comparação entre dois jogadores.
        Use elementos como caixas de texto, dropdowns, radio buttons e checkboxes para tornar a interação mais fluida.


In [ ]:
# Formulário para selecionar intervalo de tempo
selected_minute_range = st.sidebar.slider('Selecione o Intervalo de Minutos da Partida', 0, max_minute, (0, max_minute))

# Filtrar eventos no intervalo selecionado
filtered_events_in_range = filtered_events[
    (filtered_events['minute'] >= selected_minute_range[0]) &
    (filtered_events['minute'] <= selected_minute_range[1])
]
st.dataframe(filtered_events_in_range)


   9 - Implementar funcionalidades avançadas
        Utilize o Cache do Streamlit para otimizar o carregamento de dados, especialmente se estiver utilizando bases de dados grandes.
        Armazene o estado da sessão do usuário utilizando Session State, garantindo que a interação do usuário não seja perdida quando ele navegar entre páginas.


In [ ]:
@st.cache
def load_events(match_id):
    return sb.events(match_id=match_id)


   10 - Publicar o projeto
        Realize o deploy da aplicação utilizando o Streamlit Community Cloud. Verifique se o deploy foi bem-sucedido e que todas as funcionalidades estão funcionando conforme esperado.
        Compartilhe o link da aplicação publicada e o repositório no GitHub.

In [ ]:
from statsbombpy import sb

# Carregar dados de competições
competitions = sb.competitions()
st.dataframe(competitions)


In [ ]:
from mplsoccer import Pitch

# Criar o campo de futebol
pitch = Pitch()
fig, ax = pitch.draw()

# Exemplos de passes
passes = [(10, 20, 30, 40), (20, 25, 50, 60)]
for p in passes:
    pitch.arrows(p[0], p[1], p[2], p[3], ax=ax)

st.pyplot(fig)


In [5]:
st.metric(label="Gols", value=3, delta=1)
st.metric(label="Passes bem-sucedidos", value=85)


2024-10-02 01:36:44.910 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 01:36:44.911 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 01:36:44.911 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 01:36:44.911 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
intervalo = st.slider('Intervalo de tempo', 0, 90, (15, 75))


In [ ]:
@st.cache
def load_data():
    return sb.competitions()
